## 特徴量の準備
すでに提出したsubmisson_1を元に特徴量の調整や違う種類の機械学習アルゴリズムを使って予測結果を提出する


In [2]:
import numpy as np
import pandas as pd

train = pd.read_csv('/workspace/train.csv')
test = pd.read_csv('/workspace/test.csv')
gender_submission = pd.read_csv('/workspace/gender_submission.csv')

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna('S', inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['FamilySize'] = data['SibSp']+data['Parch']+1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

/tmp/ipykernel_9196/332646744.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
/tmp/ipykernel_9196/332646744.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
/tmp/ipyke

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [4]:
delete_column = ['PassengerId', 'Name', 'Ticket', 'Cabin']
data.drop(delete_column, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [5]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,3,0,22.0,1,0,7.2500,0,2,0
1,1,1,38.0,1,0,71.2833,1,2,0
2,3,1,26.0,0,0,7.9250,0,1,1
3,1,1,35.0,1,0,53.1000,0,2,0
4,3,0,35.0,0,0,8.0500,0,1,1


## sklearn
1回目の提出はロジスティック回帰を使ったが、今回はランダムフォレストを使ってみる。

### 以下２つのパラメータは過学習に注意。
* n_estuimatorsは決定木の数を指定
* max_depthは各決定木の深さを指定
random_stateはSeed値の指定で再現性を担保する

In [6]:
from sklearn.ensemble import RandomForestClassifier
cls = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [7]:
# モデルをインスタンス化したら、学習と予測はロジスティック回帰の場合と同様に実行可能
cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)

In [8]:
y_pred[:10]

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0.])

In [9]:
sub = gender_submission
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('/workspace/submission_2.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [10]:
sub['Survived'].value_counts()

Survived
0    283
1    135
Name: count, dtype: int64